In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [71]:
from numpy.random import RandomState

In [49]:
np.random.seed(42)

In [73]:
RandomState(5).permutation(14)

array([ 5,  1,  7,  2, 10, 13, 11,  4,  8,  9,  0, 12,  6,  3])

In [3]:
CV_Match = pd.read_excel("./data/kaggle/data.xlsx", sheet_name= "Match", index_col = 'Candidate_match') 
CV_Sourcing = pd.read_excel("./data/kaggle/data.xlsx", sheet_name = "Candidate", index_col = 'CV_ID')

In [32]:
CV_Sourcing

,Experience,Language Level - French
CV_ID,,
1000,NaN,NaN
1001,NaN,NaN
1002,NaN,NaN
1003,NaN,NaN
1004,NaN,NaN
...,...,...
995,NaN,NaN
996,NaN,NaN
997,NaN,NaN


Adding in the data augmentation steps as outlined here: https://docs.google.com/document/d/13W0YIM3UcS--UGs5K5s5_DHYr0cfRZC_QYr146-3AfI/edit#heading=h.4rkzkuexnsde

In [88]:
CV_Sourcing.Experience.notnull().sum()

786

In [7]:
CV_Sourcing['Language Level - French'].unique()

array([nan, 'B1', 'A1', 'C2', 'Native', 'B2', 'A2', 'C1'], dtype=object)

In [403]:
dfMerged = CV_Sourcing.merge(CV_Match, left_on=CV_Sourcing.index, right_on=CV_Match.index).rename(columns={'key_0':'CV_ID'}).sort_index()

In [404]:
statusMap = {
'A01 - Offer Preparation Started':1,
 'A02 - Offer Sent to Candidate':1,
 'A03 - Offer accepted':1,
 'A03 - Offer declined':1,
 'A03 - Process Cancelled':1,
 'A04 - Hiring Request Started':1,
 'A04b – Hiring Started / Contract signed':1,
 'A05 - Onboarding':1,
 'Approved':0,
 'CV approved':0,
 'CV dropped out':0,
 'CV refused':0,
 'CV sent to France':0,
 'Candidate dropped out':0,
 'Candidate refused':0,
 'Candidate validated':1,
 'D01 – Resignation':1,
 'Dropped out':0,
 'Matched':0,
 'Rejected':0,
 'Sent to Client':0,
 'T01- Terminated':1
}

In [405]:
dfMerged['MatchStatus'] = dfMerged['Status'].map(statusMap).astype(int)

In [406]:
dfMerged.MatchStatus.value_counts()

0    1908
1     347
Name: MatchStatus, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split

In [269]:
dfMerged

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1
...,...,...,...,...,...,...,...,...,...
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,0
2253,9836,[0-1],NaN,12443,A05 - Onboarding,2017-12-01,2017-12-01,2017-12-01,1


In [76]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMerged.iloc[:,:-1], dfMerged.iloc[:,-1], random_state=42, 
                                                test_size=.5, stratify=dfMerged.iloc[:,-1])

In [79]:
Xtrain.join(ytrain)['MatchStatus'].value_counts()

0    954
1    173
Name: MatchStatus, dtype: int64

In [80]:
Xtest.join(ytest)['MatchStatus'].value_counts()

0    954
1    174
Name: MatchStatus, dtype: int64

In [82]:
Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [96]:
baseGenderDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [224]:
# Deviations from demographic parity 
def deviation_func_gender(x, df, colName='Gender'):
    yesDF = df[(df.MatchStatus == 1)]
    noDF = df[~(df.MatchStatus == 1)]
    totalMalePercent = .5 + (x/100)
    totalMales = int(len(yesDF) * (totalMalePercent))
    
    np.random.seed(42)
    indM = np.random.choice(yesDF.index, totalMales, replace=False)
    indF = yesDF[~yesDF.index.isin(indM)].index
    
    yesDF.loc[indM, colName] = 'M'
    yesDF.loc[indF, colName] = 'F'
    
    dfNew = pd.concat([yesDF, noDF])
    return dfNew

In [292]:
# Deviations from demographic parity 
def deviation_func_race(x, df, colName='Black'):
    yesDF = df[(df.MatchStatus == 1)]
    noDF = df[(df.MatchStatus == 0)]
    totalBlackPercent = .5 + (x/100)
    totalBlacks = int(len(yesDF) * (totalBlackPercent))
    
    np.random.seed(42)
    indBlacks = np.random.choice(yesDF.index, totalBlacks, replace=False)
    indWhites = yesDF[~yesDF.index.isin(indBlacks)].index
    
    yesDF.loc[indBlacks, colName] = 0
    yesDF.loc[indWhites, colName] = 1
    
    dfNew = pd.concat([yesDF, noDF])
    return dfNew

In [238]:
deviation0 = deviation_func_gender(0, baseGenderDF)
deviation2 = deviation_func_gender(2, baseGenderDF)
deviation4 = deviation_func_gender(4, baseGenderDF)
deviation6 = deviation_func_gender(6, baseGenderDF)
deviation8 = deviation_func_gender(8, baseGenderDF)
deviation10 = deviation_func_gender(10, baseGenderDF)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [241]:
deviation0.to_csv('./data/kaggle/base-gender-dataset.csv')
deviation2.to_csv('./data/kaggle/base-gender-2-percent-dataset.csv')
deviation4.to_csv('./data/kaggle/base-gender-4-percent-dataset.csv')
deviation6.to_csv('./data/kaggle/base-gender-6-percent-dataset.csv')
deviation8.to_csv('./data/kaggle/base-gender-8-percent-dataset.csv')
deviation10.to_csv('./data/kaggle/base-gender-10-percent-dataset.csv')

In [266]:
baseGenderDF = baseGenderDF.drop('Gender', axis=1)

In [435]:
deviation0

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,M,1.0
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,F,1.0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,M,1.0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,M,1.0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,F,1.0
...,...,...,...,...,...,...,...,...,...,...
2249,9821,NaN,NaN,12961,CV refused,2018-04-09,2018-04-09,NaT,M,0.0
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,M,0.0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,M,0.0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,M,0.0


In [273]:
# Xtrain.drop('Gender', axis=1, inplace=True)
# Xtrain['Black'] = 1
Xtest.drop('Gender', axis=1, inplace=True)
Xtest['Black'] = 0

In [274]:
baseRaceDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [276]:
baseRaceDF.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        174         171                      133       174   
            1        173         169                      137       173   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         174                   174   
            1         173                   173   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    174   
            1                                    173   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                174  
            1                                173

In [293]:
racedeviation0 = deviation_func_race(0, baseRaceDF, colName='Black')
racedeviation2 = deviation_func_race(2, baseRaceDF)
racedeviation4 = deviation_func_race(4, baseRaceDF)
racedeviation6 = deviation_func_race(6, baseRaceDF)
racedeviation8 = deviation_func_race(8, baseRaceDF)
racedeviation10 = deviation_func_race(10, baseRaceDF)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [296]:
racedeviation0.to_csv('./data/kaggle/base-race-dataset.csv')
racedeviation2.to_csv('./data/kaggle/base-race-2-percent-dataset.csv')
racedeviation4.to_csv('./data/kaggle/base-race-4-percent-dataset.csv')
racedeviation6.to_csv('./data/kaggle/base-race-6-percent-dataset.csv')
racedeviation8.to_csv('./data/kaggle/base-race-8-percent-dataset.csv')
racedeviation10.to_csv('./data/kaggle/base-race-10-percent-dataset.csv')

In [294]:
racedeviation0.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        173         168                      135       173   
            1        174         172                      135       174   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         173                   173   
            1         174                   174   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    173   
            1                                    174   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                173  
            1                                174

In [295]:
racedeviation10.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        208         203                      165       208   
            1        139         137                      105       139   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         208                   208   
            1         139                   139   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    208   
            1                                    139   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                208  
            1                                139

In [316]:
deviation2.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0           F         954         205                      172       954   
            M         954         205                      158       954   
1           F         167         165                      129       167   
            M         180         175                      141       180   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0           F          954                   629   
            M          954                   650   
1           F          167                   167   
            M          180                   180   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0           F                                     659   
            M                                     670   
1           F                                     167   
            M                                     180   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                 265  
            M                                 279  
1           F                                 167  
            M                                 180

### Using unique CV_IDs for classifciation of race and gender
Adding in the correct class dataset as there many multiple duplicate entries the first time aroudn

In [354]:
uniqueCVIDs = baseGenderDF.groupby('CV_ID')[['MatchStatus']].mean().reset_index()

In [360]:
# anyone with at least one 1 in teh matchstatus is going to be a YES
yesUniqueCVIDs = uniqueCVIDs[uniqueCVIDs.MatchStatus>0]
noUniqueCVIDs = uniqueCVIDs[uniqueCVIDs.MatchStatus==0]

In [362]:
yesUniqueCVIDs.loc[:,'MatchStatus'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [407]:
dfMergedUnique = pd.concat([yesUniqueCVIDs, noUniqueCVIDs])

In [408]:
dfMerged

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1
...,...,...,...,...,...,...,...,...,...
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,0
2253,9836,[0-1],NaN,12443,A05 - Onboarding,2017-12-01,2017-12-01,2017-12-01,1


In [409]:
dfMergedUnique

,CV_ID,MatchStatus
19,369,1.0
24,374,1.0
26,376,1.0
37,402,1.0
43,431,1.0
...,...,...
1320,13281,0.0
1321,13288,0.0
1322,13319,0.0
1323,13350,0.0


In [410]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMergedUnique.iloc[:,0], dfMergedUnique.iloc[:,1], random_state=42, 
                                                test_size=.5, stratify=dfMergedUnique.iloc[:,1])

In [411]:
pd.DataFrame(Xtest).join(ytest)['MatchStatus'].value_counts()

0.0    492
1.0    171
Name: MatchStatus, dtype: int64

In [412]:
pd.DataFrame(Xtrain).join(ytrain)['MatchStatus'].value_counts()

0.0    492
1.0    170
Name: MatchStatus, dtype: int64

In [396]:
Xtest = pd.DataFrame(Xtest)
Xtrain = pd.DataFrame(Xtrain)

Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [397]:
XtestDF

,CV_ID,Gender
1070,8785,M
289,1446,M
916,6182,M
951,6570,M
650,3910,M
...,...,...
178,793,M
337,1753,M
21,371,M
551,2392,M


In [398]:
baseGenderDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [400]:
Xtrain.drop('Gender', axis=1, inplace=True)
Xtrain['Black'] = 1
Xtest.drop('Gender', axis=1, inplace=True)
Xtest['Black'] = 0

In [401]:
baseRaceDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [402]:
baseRaceDF

,CV_ID,Black,MatchStatus
0,349,1,0.0
1,350,0,0.0
2,351,1,0.0
3,352,0,0.0
4,353,0,0.0
...,...,...,...
1320,13281,1,0.0
1321,13288,0,0.0
1322,13319,0,0.0
1323,13350,1,0.0


In [414]:
dfMerged = dfMerged.drop('MatchStatus', axis=1)

In [417]:
baseGenderMerged = dfMerged.merge(baseGenderDF, left_on='CV_ID', right_on='CV_ID', how='left')

In [418]:
baseRaceMerged = dfMerged.merge(baseRaceDF, left_on='CV_ID', right_on='CV_ID', how='left')

In [429]:
deviation0 = deviation_func_gender(0, baseGenderMerged)
deviation2 = deviation_func_gender(2, baseGenderMerged)
deviation4 = deviation_func_gender(4, baseGenderMerged)
deviation6 = deviation_func_gender(6, baseGenderMerged)
deviation8 = deviation_func_gender(8, baseGenderMerged)
deviation10 = deviation_func_gender(10, baseGenderMerged)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [430]:
deviation0.to_csv('./data/kaggle/base-gender-dataset-unique-CVIDs.csv')
deviation2.to_csv('./data/kaggle/base-gender-2-percent-dataset-unique-CVIDs.csv')
deviation4.to_csv('./data/kaggle/base-gender-4-percent-dataset-unique-CVIDs.csv')
deviation6.to_csv('./data/kaggle/base-gender-6-percent-dataset-unique-CVIDs.csv')
deviation8.to_csv('./data/kaggle/base-gender-8-percent-dataset-unique-CVIDs.csv')
deviation10.to_csv('./data/kaggle/base-gender-10-percent-dataset-unique-CVIDs.csv')

In [431]:
racedeviation0 = deviation_func_race(0, baseRaceMerged, colName='Black')
racedeviation2 = deviation_func_race(2, baseRaceMerged)
racedeviation4 = deviation_func_race(4, baseRaceMerged)
racedeviation6 = deviation_func_race(6, baseRaceMerged)
racedeviation8 = deviation_func_race(8, baseRaceMerged)
racedeviation10 = deviation_func_race(10, baseRaceMerged)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [433]:
racedeviation10.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0.0         0        844          86                       57       844   
            1        833          96                       66       833   
1.0         0        346         339                      284       346   
            1        232         229                      193       232   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0.0         0         844                   553   
            1         833                   546   
1.0         0         346                   314   
            1         232                   213   

                   4. Client initial validation date  \
MatchStatus Black                                      
0.0         0                                    583   
            1                                    565   
1.0         0                                    315   
            1                                    213   

                   7. Client final approval date  
MatchStatus Black                                 
0.0         0                                237  
            1                                238  
1.0         0                                249  
            1                                167

In [434]:
racedeviation0.to_csv('./data/kaggle/base-race-dataset-unique-CVIDs.csv')
racedeviation2.to_csv('./data/kaggle/base-race-2-percent-dataset-unique-CVIDs.csv')
racedeviation4.to_csv('./data/kaggle/base-race-4-percent-dataset-unique-CVIDs.csv')
racedeviation6.to_csv('./data/kaggle/base-race-6-percent-dataset-unique-CVIDs.csv')
racedeviation8.to_csv('./data/kaggle/base-race-8-percent-dataset-unique-CVIDs.csv')
racedeviation10.to_csv('./data/kaggle/base-race-10-percent-dataset-unique-CVIDs.csv')

### Using values of Experience and Language Level with deviations on race and gender

In [510]:
expLangDFNoNull = dfMerged[['Experience','Language Level - French']].dropna()
dfMergedNoNulls = dfMerged.iloc[expLangDFNoNull.index,:]

In [511]:
dfMergedNoNulls['MatchStatus'] = dfMergedNoNulls['Status'].map(statusMap).astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [512]:
dfMergedNoNulls.MatchStatus.value_counts()

0    304
1    269
Name: MatchStatus, dtype: int64

In [ ]:
dfMergedNoNulls.groupby

In [513]:
dfMergedNoNulls

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1
...,...,...,...,...,...,...,...,...,...
2242,9820,[8+],A1,12063,Matched,NaT,NaT,NaT,0
2243,9820,[8+],A1,12808,Dropped out,2018-03-29,2018-04-03,NaT,0
2244,9820,[8+],A1,12829,Dropped out,2018-03-29,2018-04-03,2018-04-03,0
2245,9820,[8+],A1,12832,CV refused,2018-03-29,2018-03-29,NaT,0


In [514]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMergedNoNulls.iloc[:,:-1], dfMergedNoNulls.iloc[:,-1], random_state=42, 
                                                test_size=.5, stratify=dfMergedNoNulls.iloc[:,-1])

In [515]:
Xtrain.join(ytrain)['MatchStatus'].value_counts()

0    152
1    134
Name: MatchStatus, dtype: int64

In [516]:
Xtest.join(ytest)['MatchStatus'].value_counts()

0    152
1    135
Name: MatchStatus, dtype: int64

In [517]:
Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [518]:
baseGenderDFNoNulls = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()
# not right

In [519]:
uniqueCVIDsNoNulls = baseGenderDFNoNulls.groupby('CV_ID')[['MatchStatus']].mean().reset_index()

In [531]:
uniqueCVIDsNoNulls[uniqueCVIDsNoNulls.MatchStatus>0]

,CV_ID,MatchStatus
3,369,1.000000
4,374,0.250000
5,376,0.200000
8,402,0.500000
12,460,0.500000
...,...,...
290,12612,0.333333
292,12732,1.000000
293,12736,1.000000
294,12737,1.000000


In [527]:
# anyone with at least one 1 in teh matchstatus is going to be a YES
yesUniqueCVIDsNoNulls = uniqueCVIDsNoNulls[uniqueCVIDsNoNulls.MatchStatus>0]['CV_ID']
noUniqueCVIDsNoNulls = uniqueCVIDsNoNulls[uniqueCVIDsNoNulls.MatchStatus==0]['CV_ID']

In [629]:
# Jesse's additions
dfMergedNoNulls['response_time'] = dfMergedNoNulls['4. Client initial validation date'] - dfMergedNoNulls['4. CV sent to Client']
dfMergedNoNulls['decision_time'] = dfMergedNoNulls['7. Client final approval date'] - dfMergedNoNulls['4. Client initial validation date'] 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [630]:
dfMergedNoNulls

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus,response_time,decision_time
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1,2 days,19 days
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0,NaT,NaT
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0,0 days,NaT
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0,0 days,NaT
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1,0 days,38 days
...,...,...,...,...,...,...,...,...,...,...,...
2242,9820,[8+],A1,12063,Matched,NaT,NaT,NaT,0,NaT,NaT
2243,9820,[8+],A1,12808,Dropped out,2018-03-29,2018-04-03,NaT,0,5 days,NaT
2244,9820,[8+],A1,12829,Dropped out,2018-03-29,2018-04-03,2018-04-03,0,5 days,0 days
2245,9820,[8+],A1,12832,CV refused,2018-03-29,2018-03-29,NaT,0,0 days,NaT


In [632]:
dfMergedNoNullsAtLeastOneYes = pd.DataFrame(yesUniqueCVIDsNoNulls).merge(dfMergedNoNulls, left_on='CV_ID', right_on='CV_ID', how='left' ).set_index('CV_ID')

In [633]:
dfMergedNoNullsAllNos = pd.DataFrame(noUniqueCVIDsNoNulls).merge(dfMergedNoNulls, left_on='CV_ID', right_on='CV_ID', how='left' ).set_index('CV_ID')

In [634]:
# Deviations from demographic parity 
def deviation_func_gender_no_nulls(x, yesSeries, noSeries, colName='Gender'):
    # yesDF = df[(df.MatchStatus == 1)]
    # noDF = df[~(df.MatchStatus == 1)]
    totalMalePercent = .5 + (x/100)
    totalMales = int(len(yesSeries) * (totalMalePercent))
    totalMalesNo = int(len(noSeries) * (.5))
    
    np.random.seed(42)
    indM = np.random.choice(yesSeries.values, totalMales, replace=False)
    
    dfNewYes = dfMergedNoNullsAtLeastOneYes.copy()
    dfNewYes.loc[indM, colName] = 'M'
    dfNewYes[colName] = dfNewYes[colName].fillna('F')
    
    indMNo = np.random.choice(noSeries.values, totalMalesNo, replace=False)
    dfNewNo = dfMergedNoNullsAllNos.copy()
    dfNewNo.loc[indMNo, colName] = 'M'
    dfNewNo[colName] = dfNewNo[colName].fillna('F')
        
    
    dfNew = pd.concat([dfNewYes, dfNewNo])
    return dfNew

In [635]:
# Deviations from racist parity 
def deviation_func_race_no_nulls(x, yesSeries, noSeries, colName='Black'):
    totalWhitePercent = .5 + (x/100)
    totalWhite = int(len(yesSeries) * (totalWhitePercent))
    totalWhitesNo = int(len(noSeries) * (.5))
    
    np.random.seed(42)
    indW = np.random.choice(yesSeries.values, totalWhite, replace=False)
    
    dfNewYes = dfMergedNoNullsAtLeastOneYes.copy()
    dfNewYes.loc[indW, colName] = 0
    dfNewYes[colName] = dfNewYes[colName].fillna(1)
    
    indWNo = np.random.choice(noSeries.values, totalWhitesNo, replace=False)
    dfNewNo = dfMergedNoNullsAllNos.copy()
    dfNewNo.loc[indWNo, colName] = 0
    dfNewNo[colName] = dfNewNo[colName].fillna(1)
        
    
    dfNew = pd.concat([dfNewYes, dfNewNo])
    return dfNew

In [602]:
x= deviation_func_gender_no_nulls(0, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
y = deviation_func_race_no_nulls(0, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)

In [636]:
deviation0 = deviation_func_gender_no_nulls(0, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
deviation2 = deviation_func_gender_no_nulls(2, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
deviation4 = deviation_func_gender_no_nulls(4, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
deviation6 = deviation_func_gender_no_nulls(6, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
deviation8 = deviation_func_gender_no_nulls(8, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
deviation10 = deviation_func_gender_no_nulls(10, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)

In [619]:
deviation10.groupby(['MatchStatus','Gender']).count()

Experience  Language Level - French  Match_ID  Status  \
MatchStatus Gender                                                          
0           F              147                      147       147     147   
            M              157                      157       157     157   
1           F              108                      108       108     108   
            M              161                      161       161     161   

                    4. CV sent to Client  4. Client initial validation date  \
MatchStatus Gender                                                            
0           F                         99                                 99   
            M                        112                                115   
1           F                        108                                108   
            M                        161                                161   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                  33  
            M                                  44  
1           F                                 108  
            M                                 161

In [639]:
# need to add this

deviation0.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang.csv')
deviation2.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang-2-percent.csv')
deviation4.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang-4-percent.csv')
deviation6.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang-6-percent.csv')
deviation8.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang-8-percent.csv')
deviation10.to_csv('./data/kaggle/dataExpLang/base-gender-dataset-with-exp-lang-10-percent.csv')

In [640]:
racedeviation0 = deviation_func_race_no_nulls(0, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
racedeviation2 = deviation_func_race_no_nulls(2, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
racedeviation4 = deviation_func_race_no_nulls(4, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
racedeviation6 = deviation_func_race_no_nulls(6, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
racedeviation8 = deviation_func_race_no_nulls(8, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)
racedeviation10 = deviation_func_race_no_nulls(10, yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls)

In [641]:
racedeviation6.groupby(['MatchStatus','Black']).count()

Experience  Language Level - French  Match_ID  Status  \
MatchStatus Black                                                          
0           0.0           150                      150       150     150   
            1.0           154                      154       154     154   
1           0.0           151                      151       151     151   
            1.0           118                      118       118     118   

                   4. CV sent to Client  4. Client initial validation date  \
MatchStatus Black                                                            
0           0.0                     106                                109   
            1.0                     105                                105   
1           0.0                     151                                151   
            1.0                     118                                118   

                   7. Client final approval date  response_time  decision_time  
MatchStatus Black                                                               
0           0.0                               40            104             39  
            1.0                               37            102             36  
1           0.0                              151            151            151  
            1.0                              118            118            118

In [434]:
# need to add these

racedeviation0.to_csv('./data/kaggle/base-race-dataset-unique-CVIDs.csv')
racedeviation2.to_csv('./data/kaggle/base-race-2-percent-dataset-unique-CVIDs.csv')
racedeviation4.to_csv('./data/kaggle/base-race-4-percent-dataset-unique-CVIDs.csv')
racedeviation6.to_csv('./data/kaggle/base-race-6-percent-dataset-unique-CVIDs.csv')
racedeviation8.to_csv('./data/kaggle/base-race-8-percent-dataset-unique-CVIDs.csv')
racedeviation10.to_csv('./data/kaggle/base-race-10-percent-dataset-unique-CVIDs.csv')

In [598]:
x

,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus,Gender
CV_ID,,,,,,,,,
369,[8+],A1,498,A05 - Onboarding,2018-02-22,2018-02-22,2018-02-23,1,F
374,[8+],A1,11653,CV refused,2018-01-09,2018-01-10,NaT,0,F
374,[8+],A1,11657,CV refused,2018-01-09,2018-01-23,NaT,0,F
374,[8+],A1,11769,Approved,NaT,NaT,NaT,0,F
374,[8+],A1,385,A03 - Offer declined,2018-01-30,2018-01-30,2018-02-16,1,F
...,...,...,...,...,...,...,...,...,...
13242,[2-4],A1,13314,CV sent to France,NaT,NaT,NaT,0,M
13242,[2-4],A1,13466,CV sent to France,NaT,NaT,NaT,0,M
13271,[0-1],Native,13380,Matched,NaT,NaT,NaT,0,M


In [599]:
x.groupby(['MatchStatus','Gender']).count()

Experience  Language Level - French  Match_ID  Status  \
MatchStatus Gender                                                          
0           F              164                      164       164     164   
            M              140                      140       140     140   
1           F              134                      134       134     134   
            M              135                      135       135     135   

                    4. CV sent to Client  4. Client initial validation date  \
MatchStatus Gender                                                            
0           F                        114                                115   
            M                         97                                 99   
1           F                        134                                134   
            M                        135                                135   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                  43  
            M                                  34  
1           F                                 134  
            M                                 135

In [603]:
y

,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus,Black
CV_ID,,,,,,,,,
369,[8+],A1,498,A05 - Onboarding,2018-02-22,2018-02-22,2018-02-23,1,1.0
374,[8+],A1,11653,CV refused,2018-01-09,2018-01-10,NaT,0,1.0
374,[8+],A1,11657,CV refused,2018-01-09,2018-01-23,NaT,0,1.0
374,[8+],A1,11769,Approved,NaT,NaT,NaT,0,1.0
374,[8+],A1,385,A03 - Offer declined,2018-01-30,2018-01-30,2018-02-16,1,1.0
...,...,...,...,...,...,...,...,...,...
13242,[2-4],A1,13314,CV sent to France,NaT,NaT,NaT,0,0.0
13242,[2-4],A1,13466,CV sent to France,NaT,NaT,NaT,0,0.0
13271,[0-1],Native,13380,Matched,NaT,NaT,NaT,0,0.0


In [604]:
y.groupby(['MatchStatus','Black']).count()

Experience  Language Level - French  Match_ID  Status  \
MatchStatus Black                                                          
0           0.0           140                      140       140     140   
            1.0           164                      164       164     164   
1           0.0           135                      135       135     135   
            1.0           134                      134       134     134   

                   4. CV sent to Client  4. Client initial validation date  \
MatchStatus Black                                                            
0           0.0                      97                                 99   
            1.0                     114                                115   
1           0.0                     135                                135   
            1.0                     134                                134   

                   7. Client final approval date  
MatchStatus Black                                 
0           0.0                               34  
            1.0                               43  
1           0.0                              135  
            1.0                              134

In [551]:
halfSplitAtLeastOneYes = len(dfMergedNoNullsAtLeastOneYes)/2
np.random.choice(dfMergedNoNullsAtLeastOneYes.CV_ID.unique(), int(halfSplitAtLeastOneYes), replace=False)

array([ 5372,  1045, 12612,  4398,  2095,  1813,  2030,   460,  1071,
        4987,  1812,  2421,  8190,  2299,  4646,  9091, 12521,   693,
        3584,  2005,  2247,  5109,   565,  2501,  1635,  1797,  1695,
        1384,  8047,   594,  6520,  3748,  7856,  3829,   812,  5699,
        2359,  5445,  1822,  8218,  1747,  1882,  7887,  2099,  5603,
        4917,  1909,  6012,  1582,  5149,  1317,  1025,  6333,  3898,
        4289,  1799,  2094,  1988,  5534,  4428,  6022,  4544,  1619,
        6518,  1814,   899,  6094, 12611,  5946,  1305,  1943,  2351,
        1624,  2105,  2324,  1886,  2115,  6300,  1613,  1948,  7539,
        1310,  4764,  8649,  5956,  4620,  6308,  7319,  2047,  3087,
        3763,  2070,  6918,  7010,  2358,   573,  4692,  6196,  7181,
        1984,  5906,  2353,  1491,  1134,   552,  7318,  3703,  5616,
        3497,  1889,  2046,  3096,  2956,  2420,   698,  2360,  8834,
        4806,  3614,  1386, 12737,  2518,   376,  1844,  4265,  6236,
        6516,  2784,

In [548]:
# yesUniqueCVIDsNoNulls.loc[:,'MatchStatus'] = 1

In [529]:
dfMergedUniqueNoNulls = pd.concat([yesUniqueCVIDsNoNulls, noUniqueCVIDsNoNulls])

In [530]:
dfMergedUniqueNoNulls

3        369
4        374
5        376
8        402
12       460
       ...  
311    13214
312    13242
313    13271
314    13281
315    13288
Name: CV_ID, Length: 316, dtype: int64

In [525]:
dfMergedUniqueNoNulls.MatchStatus.value_counts()

1.000000    153
0.000000     53
0.500000     50
0.333333     36
0.250000     11
0.200000      6
0.166667      4
0.666667      2
0.400000      1
Name: MatchStatus, dtype: int64

In [495]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMergedUniqueNoNulls.iloc[:,0], dfMergedUniqueNoNulls.iloc[:,1], random_state=42, 
                                                test_size=.5, stratify=dfMergedUniqueNoNulls.iloc[:,1])

In [496]:
pd.DataFrame(Xtest).join(ytest)['MatchStatus'].value_counts()

1.0    131
0.0     27
Name: MatchStatus, dtype: int64

In [497]:
pd.DataFrame(Xtrain).join(ytrain)['MatchStatus'].value_counts()

1.0    132
0.0     26
Name: MatchStatus, dtype: int64

In [498]:
Xtest = pd.DataFrame(Xtest)
Xtrain = pd.DataFrame(Xtrain)

Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [499]:
Xtest

,CV_ID,Gender
91,1913,M
290,12612,M
153,3497,M
292,12732,M
139,2501,M
...,...,...
8,402,M
200,5189,M
168,4031,M
42,951,M


In [500]:
baseGenderDFNoNulls = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [501]:
Xtrain.drop('Gender', axis=1, inplace=True)
Xtrain['Black'] = 1
Xtest.drop('Gender', axis=1, inplace=True)
Xtest['Black'] = 0

In [502]:
baseRaceDFNoNulls = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [506]:
baseGenderDFNoNulls.groupby(['MatchStatus','Gender']).count()

CV_ID
MatchStatus Gender       
0.0         F          26
            M          27
1.0         F         132
            M         131

In [505]:
baseRaceDFNoNulls.groupby(['MatchStatus','Black']).count()

CV_ID
MatchStatus Black       
0.0         0         27
            1         26
1.0         0        131
            1        132

In [509]:
dfMergedNoNulls.merge(baseGenderDFNoNulls, left_on='CV_ID', right_on='CV_ID', how='left')

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus_x,Gender,MatchStatus_y
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1,F,1.0
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0,M,1.0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0,M,1.0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0,M,1.0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1,M,1.0
...,...,...,...,...,...,...,...,...,...,...,...
568,9820,[8+],A1,12063,Matched,NaT,NaT,NaT,0,F,0.0
569,9820,[8+],A1,12808,Dropped out,2018-03-29,2018-04-03,NaT,0,F,0.0
570,9820,[8+],A1,12829,Dropped out,2018-03-29,2018-04-03,2018-04-03,0,F,0.0
571,9820,[8+],A1,12832,CV refused,2018-03-29,2018-03-29,NaT,0,F,0.0


In [414]:
dfMerged = dfMerged.drop('MatchStatus', axis=1)

In [417]:
baseGenderMerged = dfMerged.merge(baseGenderDF, left_on='CV_ID', right_on='CV_ID', how='left')

In [418]:
baseRaceMerged = dfMerged.merge(baseRaceDF, left_on='CV_ID', right_on='CV_ID', how='left')

In [429]:
deviation0 = deviation_func_gender(0, baseGenderMerged)
deviation2 = deviation_func_gender(2, baseGenderMerged)
deviation4 = deviation_func_gender(4, baseGenderMerged)
deviation6 = deviation_func_gender(6, baseGenderMerged)
deviation8 = deviation_func_gender(8, baseGenderMerged)
deviation10 = deviation_func_gender(10, baseGenderMerged)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [430]:
deviation0.to_csv('./data/kaggle/base-gender-dataset-unique-CVIDs.csv')
deviation2.to_csv('./data/kaggle/base-gender-2-percent-dataset-unique-CVIDs.csv')
deviation4.to_csv('./data/kaggle/base-gender-4-percent-dataset-unique-CVIDs.csv')
deviation6.to_csv('./data/kaggle/base-gender-6-percent-dataset-unique-CVIDs.csv')
deviation8.to_csv('./data/kaggle/base-gender-8-percent-dataset-unique-CVIDs.csv')
deviation10.to_csv('./data/kaggle/base-gender-10-percent-dataset-unique-CVIDs.csv')

In [431]:
racedeviation0 = deviation_func_race(0, baseRaceMerged, colName='Black')
racedeviation2 = deviation_func_race(2, baseRaceMerged)
racedeviation4 = deviation_func_race(4, baseRaceMerged)
racedeviation6 = deviation_func_race(6, baseRaceMerged)
racedeviation8 = deviation_func_race(8, baseRaceMerged)
racedeviation10 = deviation_func_race(10, baseRaceMerged)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [433]:
racedeviation10.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0.0         0        844          86                       57       844   
            1        833          96                       66       833   
1.0         0        346         339                      284       346   
            1        232         229                      193       232   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0.0         0         844                   553   
            1         833                   546   
1.0         0         346                   314   
            1         232                   213   

                   4. Client initial validation date  \
MatchStatus Black                                      
0.0         0                                    583   
            1                                    565   
1.0         0                                    315   
            1                                    213   

                   7. Client final approval date  
MatchStatus Black                                 
0.0         0                                237  
            1                                238  
1.0         0                                249  
            1                                167

In [434]:
racedeviation0.to_csv('./data/kaggle/base-race-dataset-unique-CVIDs.csv')
racedeviation2.to_csv('./data/kaggle/base-race-2-percent-dataset-unique-CVIDs.csv')
racedeviation4.to_csv('./data/kaggle/base-race-4-percent-dataset-unique-CVIDs.csv')
racedeviation6.to_csv('./data/kaggle/base-race-6-percent-dataset-unique-CVIDs.csv')
racedeviation8.to_csv('./data/kaggle/base-race-8-percent-dataset-unique-CVIDs.csv')
racedeviation10.to_csv('./data/kaggle/base-race-10-percent-dataset-unique-CVIDs.csv')

In [428]:
deviation2.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0.0         F         833          96                       66       833   
            M         844          86                       57       844   
1.0         F         278         275                      233       278   
            M         300         293                      244       300   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0.0         F          833                   546   
            M          844                   553   
1.0         F          278                   255   
            M          300                   272   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0.0         F                                     565   
            M                                     583   
1.0         F                                     255   
            M                                     273   

                    7. Client final approval date  
MatchStatus Gender                                 
0.0         F                                 238  
            M                                 237  
1.0         F                                 200  
            M                                 216

In [424]:
deviation0.groupby(['CV_ID','Gender']).count()

,,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
CV_ID,Gender,,,,,,,,
349,F,4,0,4,4,2,3,1,4
350,M,1,0,1,1,0,0,0,1
351,F,1,0,1,1,0,0,0,1
352,M,6,6,6,6,3,4,1,6
353,M,2,2,2,2,1,1,0,2
...,...,...,...,...,...,...,...,...,...
13281,F,1,1,1,1,0,0,0,1
13288,M,1,1,1,1,0,0,0,1
13319,M,1,0,1,1,0,0,0,1


In [359]:
baseRaceDF[(baseRaceDF.CV_ID == 402) ]

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Black,MatchStatus
1081,402,[0-1],A1,11091,A03 - Offer declined,2017-07-27,2017-07-27,2017-08-08,1,1
1082,402,[0-1],A1,11585,Rejected,NaT,NaT,NaT,1,0
1083,402,[0-1],A1,11587,CV refused,2017-12-12,NaT,NaT,1,0
1084,402,[0-1],A1,11955,A03 - Offer declined,2018-03-23,2018-03-23,2017-08-08,1,1


In [313]:
baseGenderDF.groupby(['CV_ID','Experience','Language Level - French','Status']).nunique()

CV_ID  Experience  \
CV_ID Experience Language Level - French Status                                 
352   [2-4]      B2                      CV refused             1           1   
                                         Candidate refused      1           1   
                                         Dropped out            1           1   
                                         Matched                1           1   
                                         Rejected               1           1   
...                                                           ...         ...   
13242 [2-4]      A1                      CV refused             1           1   
                                         CV sent to France      1           1   
13271 [0-1]      Native                  Matched                1           1   
13281 [8+]       A2                      Rejected               1           1   
13288 [8+]       Native                  Matched                1           1   

                                                            Language Level - French  \
CV_ID Experience Language Level - French Status                                       
352   [2-4]      B2                      CV refused                               1   
                                         Candidate refused                        1   
                                         Dropped out                              1   
                                         Matched                                  1   
                                         Rejected                                 1   
...                                                                             ...   
13242 [2-4]      A1                      CV refused                               1   
                                         CV sent to France                        1   
13271 [0-1]      Native                  Matched                                  1   
13281 [8+]       A2                      Rejected                                 1   
13288 [8+]       Native                  Matched                                  1   

                                                            Match_ID  Status  \
CV_ID Experience Language Level - French Status                                
352   [2-4]      B2                      CV refused                2       1   
                                         Candidate refused         1       1   
                                         Dropped out               1       1   
                                         Matched                   1       1   
                                         Rejected                  1       1   
...                                                              ...     ...   
13242 [2-4]      A1                      CV refused                1       1   
                                         CV sent to France         3       1   
13271 [0-1]      Native                  Matched                   1       1   
13281 [8+]       A2                      Rejected                  1       1   
13288 [8+]       Native                  Matched                   1       1   

                                                            4. CV sent to Client  \
CV_ID Experience Language Level - French Status                                    
352   [2-4]      B2                      CV refused                            2   
                                         Candidate refused                     1   
                                         Dropped out                           0   
                                         Matched                               0   
                                         Rejected                              0   
...                                                                          ...   
13242 [2-4]      A1                      CV refused                            1   
                                         CV sent to France                   

In [334]:
x = baseGenderDF[baseGenderDF.MatchStatus == 1]
x.groupby('CV_ID').count()

,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
CV_ID,,,,,,,,
369,1,1,1,1,1,1,1,1
374,1,1,1,1,1,1,1,1
376,1,1,1,1,1,1,1,1
402,2,2,2,2,2,2,2,2
431,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
12737,1,1,1,1,1,1,1,1
12757,1,0,1,1,1,1,1,1
12983,0,0,1,1,1,1,1,1


In [344]:
x.groupby('CV_ID')['MatchStatus'].count()[x.groupby('CV_ID')['MatchStatus'].count() > 1]

CV_ID
402      2
654      2
1813     2
2005     2
4398     2
12340    2
Name: MatchStatus, dtype: int64

In [353]:
x[x.CV_ID == 4398]

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
1143,4398,[8+],A1,10836,A03 - Process Cancelled,2017-05-23,2017-05-24,2017-06-06,1
1145,4398,[8+],A1,11706,A03 - Process Cancelled,2018-01-23,2018-03-07,2017-06-06,1


In [349]:
x.CV_ID in [654, 402, 1813, 2005]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Rough Work

In [110]:
# CV_Sourcing['Gender'] = np.random.choice(['M','F'], 10099, p=[0.65, 0.35])

In [112]:
# CV_Sourcing['Race'] = np.random.choice(['American Indian or Alaska Native','Asian',
#                                        'Black or African American','Native Hawaiian or Other Pacific Islander',
#                                         'White', 'NA'], 10099, p=[0.05, 0.3, .18, .07, .35, .05])

In [116]:
# CV_Sourcing['Parent'] = np.random.choice([1,0], 10099, p=[0.30, 0.7])

In [129]:
# CV_Sourcing['City'] = np.random.choice(['Atlanta, GA','San Francisco, CA',
#                                        'Chicago, IL','New York City, NY',
#                                         'Houston, TX', 'Dallas, TX', 
#                                         'Salt Lake City, UT', 'Miami, FL',
#                                        'Los Angeles, CA','Seattle, WA'], 10099, 
#                                        p=[0.07, 0.18, .09, .11, .07, .1, .04, .12, .1, .12])

In [131]:
# CV_Sourcing['Occupation'] = np.random.choice(['Software Developer','Administrative Assistant',
#                                        'Customer Service','Cook',
#                                         'High-skilled Sales', 'Low-skilled Sales'], 10099, 
#                                        p=[0.43, 0.12, .2, .12, .1, .03])

In [134]:
# CV_Sourcing.to_excel('./data/kaggle/augmented_data.xlsx')